In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from space_env import Satellite, Propagator

print("Entorno cargado correctamente.")

>>> [SUCCESS] GMAT LOADED
Entorno cargado correctamente.


In [3]:
# Instanciamos el satelite (esto crea el objeto Spacecraft en GMAT)
sat = Satellite("Saocom")

# Configuramos una orbita circular a 700 km de altura (LEO)
# SMA = 6371 (Radio Tierra) + 700 = 7071 km
sat.set_keplerian(
    sma=7071, 
    ecc=0.0001, 
    inc=98.2, 
    raan=0, 
    aop=0, 
    ta=0
)

print(f"Satelite '{sat.name}' configurado.")

Satelite 'Saocom' configurado.


In [ ]:
# 1. Definir la fidelidad de la física
config_leo = {
    'gravity': 'Earth',
    'degree': 2,    # Activa J2
    'order': 0,
    'drag': True    # Resistencia atmosférica (importante en LEO)
}

# 2. Instanciar el propagador (esto crea el ForceModel y el Propagator en GMAT)
prop = Propagator("MyPropagator", config=config_leo)

In [ ]:
# 5 horas en segundos
duracion_sim = 5 * 3600 

print("Propagando...")
# Esto devuelve un array de numpy: [tiempo, x, y, z]
trayectoria = prop.run(sat, duration_sec=duracion_sim, step_size=60)

print(f"¡Listo! Se generaron {trayectoria.shape[0]} puntos de datos.")

In [ ]:
--> moved to visualizer.py

# Llamada final:
figura = plot_ground_track(trayectoria)
figura.show()

In [ ]:
--> moved to visualizer.py

# Ejecución
fig_2d = graficar_2d_plotly(trayectoria)
fig_2d.show()

# From Celestrak

In [4]:
from satCatalog import request_celestrak_data
from space_env import TLEHandler

In [5]:
# Request Satellites to Celestrak
nusat_list = request_celestrak_data("NUSAT", "NAME")
nusat45=nusat_list[-1]
handler = TLEHandler.from_json(nusat45) # To propagate with SGP4

In [6]:
handler.name.split()[0]

'NUSAT-45'

In [7]:
# New Satellite
sat_name = handler.name.split()[0]
# 3. Get the state at the TLE's specific Epoch 
state = handler.get_state_at() 


In [8]:
# Initialize GMAT Satellite and set the state
sat_nusat = Satellite(sat_name)
sat_nusat.set_cartesian(
    epoch_astropy=state['epoch'],
    pos=state['pos'],
    vel=state['vel']
)

In [9]:
# 1. Interval of propagation (eg: 1 orbit ~ 90 min = 5400 sec)
duration = 16*5400 
step = 60 # dates every 60 sec

# 2. Instantiate propagator
prop = Propagator(name="HighPrecisionProp")

# 3. Execute propagation
# Esto devolverá un array de numpy con [tiempo, x, y, z]
trajectory_data = prop.run(sat_nusat, duration, step_size=step)

print(f"Propagación completada. Puntos calculados: {len(trajectory_data)}")
print(f"Última posición (km): {trajectory_data[-1, 1:]}")


GravityField  

   Field                                   Type   Value
   --------------------------------------------------------

   Degree                               Integer   4
   Order                                Integer   4
   StmLimit                             Integer   100
   PotentialFile                       Filename   JGM2.cof
   TideFile                            Filename   
   TideModel                             String   None

Propagación completada. Puntos calculados: 1440
Última posición (km): [  919.09556568 -6477.60848376 -2103.9009526 ]


In [18]:
from visualizer import graficar_2d_plotly, plot_ground_track_with_access
# Ejecución
fig_2d = graficar_2d_plotly(trajectory_data)
fig_2d.show()

# Access Analysis

In [11]:
import ground_segment as gs

In [20]:
bsas = gs.Site("BuenosAires", -34.6, -58.48, 25)
matera = gs.Site("Matera", 40.66, 16.6, 100)

In [ ]:
import access_manager as am

In [21]:
acc1 = am.AccessManager()
acc2 = am.AccessManager()
access_bsas = acc1.calculate_access(trajectory_data, state['epoch'], bsas)
access_matera = acc1.calculate_access(trajectory_data, state['epoch'], matera)

In [22]:
visual_data = [{'site': bsas, 'passes': access_bsas}, {'site': matera, 'passes': access_matera}]
fig = plot_ground_track_with_access(trajectory_data, state['epoch'], visual_data)
fig.show()